In [ ]:
# 정리

# train에는 10000개의 데이터가 있고, test에는 2062개의 데이터가 있다.
# train에는 18개의 컬럼이 있고, test에는 17개의 컬럼이 있다.
# 모든값 결측치 없음

# 예측해야할 컬럼 '채무 불이행 여부'
# 약 2:1 비율. 균형 잡힌 데이터라고 할 수는 없지만, 극단적으로 불균형한 데이터도 아님
# 이 정도의 비율 차이는 비교적 수용 가능한 수준으로 간주

# '체납 세금 압류 횟수' 0의 비율이 0.8826, 그 이외의 비율 0.1174
# 0이면 0, 1이상이면 1 범주 순서형 데이터 커스텀 피쳐 추가


# 신용 문제 발생 횟수 0의 비율 0.6621, 1의 비율 0.2149, 그 이외의 비율 0.123
# 0이면 0, 1이면 1, 2이상이면 2 범주 순서형 데이터 커스텀 피쳐 추가


# 개인 파산 횟수 0의 비율 0.7886, 1의 비율 0.1268, 그 이외의 비율 0.0846
# 0이면 0, 1이면 1, 2이상이면 2 범주 순서형 데이터 커스텀 피쳐 추가


# 나머지 수치형 데이터 비대칭적이다.
# 수치형 데이터 Transfomation -> Scaling 작업 필요
# Transform 방법, Scaling 방법 여러가지 경우의 수
# 베이스모델은 PowerTransformer yeo-johnson, MinMaxScaler

# 트랜스폼 적용 후 이상치 탐지
# 사용할 이상치: '최대 신용한도'
# '최대 신용한도_is_outlier' 생성 후 표시
# 원래 컬럼의 이상치를 중앙값으로 대체체
# 나머지 이상치들은 비율이 매우 적기 때문에 삭제해도 무방하다.
# 이상치 버릴거: 연간 소득, 개설된 신용계좌 수, 신용 거래 연수, 마지막 연체 이후 경과 개월 수, 현재 대출 잔액, 현재 미상환 신용액, 월 상환 부채액, 신용 점수
# 이후 MinMaxScaling

# 다중공선성 문제가 발생할 수 있는 변수들이 없음

# '주거 형태'는 범주 명목형 데이터 -> one hot 인코딩
# 주택 담보 대출 (비거주 중) 데이터 매우 불충분

# '현재 직장 근속 연수'는 범주 순서형 데이터 -> 라벨 인코딩
# 10년 이상 데이터가 다수, 나머지 데이터 비율은 비슷


# 대출 목적 부채통합의 비율 0.7294, 그 이외의 비율 0.2706
# 부채통합이면 0, 아니면 1 범주 명목형 데이터 커스텀 피쳐 추가


# 대출 상환 데이터는 범주 명목형 데이터
# 단기 상환    0.6975, 장기 상환    0.3025




In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import koreanize_matplotlib
import matplotlib.pyplot as plt

In [8]:
# 데이터 가져오기
train_df = pd.read_csv('./data/train.csv')
test_df = pd.read_csv('./data/test.csv')

In [9]:
# 사용할 피쳐와 타겟 데이터
features = []
target = '채무 불이행 여부'

In [13]:
# '체납 세금 압류 횟수' 0의 비율이 0.8826, 그 이외의 비율 0.1174
# 0이면 0, 1이상이면 1인인 범주 순서형 데이터 커스텀 피쳐 추가
# 데이터 불균형

train_df['체납 세금 압류 여부'] = train_df['체납 세금 압류 횟수'].apply(lambda x: 0 if x == 0 else 1)
test_df['체납 세금 압류 여부'] = test_df['체납 세금 압류 횟수'].apply(lambda x: 0 if x == 0 else 1)

In [18]:
train_df['체납 세금 압류 여부'].value_counts(), test_df['체납 세금 압류 여부'].value_counts()

(체납 세금 압류 여부
 0    8826
 1    1174
 Name: count, dtype: int64,
 체납 세금 압류 여부
 0    2012
 1      50
 Name: count, dtype: int64)

In [ ]:
# 피쳐 추가
features.append('체납 세금 압류 여부')

In [ ]:
# 신용 문제 발생 횟수 0의 비율 0.6621, 1의 비율 0.2149, 그 이외의 비율 0.123
# 0이면 0, 1이면 1, 2이상이면 2 범주 순서형 데이터 커스텀 피쳐 추가

train_df['신용 문제 범주'] = train_df['신용 문제 발생 횟수'].apply(lambda x: 0 if x == 0 else (1 if x == 1 else 2))
test_df['신용 문제 범주'] = test_df['신용 문제 발생 횟수'].apply(lambda x: 0 if x == 0 else (1 if x == 1 else 2))

In [21]:
train_df['신용 문제 범주'].value_counts(), test_df['신용 문제 범주'].value_counts()

(신용 문제 범주
 0    6621
 1    2149
 2    1230
 Name: count, dtype: int64,
 신용 문제 범주
 0    1756
 1     255
 2      51
 Name: count, dtype: int64)

In [22]:
# 피쳐 추가
features.append('신용 문제 범주')

In [ ]:
# 개인 파산 횟수 0의 비율 0.7886, 1의 비율 0.1268, 그 이외의 비율 0.0846
# 0이면 0, 1이면 1, 2이상이면 2 범주 순서형 데이터 커스텀 피쳐 추가

train_df['개인 파산 범주'] = train_df['개인 파산 횟수'].apply(lambda x: 0 if x == 0 else (1 if x == 1 else 2))
test_df['개인 파산 범주'] = test_df['개인 파산 횟수'].apply(lambda x: 0 if x == 0 else (1 if x == 1 else 2))

In [24]:
train_df['개인 파산 범주'].value_counts(), test_df['개인 파산 범주'].value_counts()

(개인 파산 범주
 0    7886
 1    1268
 2     846
 Name: count, dtype: int64,
 개인 파산 범주
 0    1837
 1     215
 2      10
 Name: count, dtype: int64)

In [25]:
# 피쳐 추가
features.append('개인 파산 범주')

In [ ]:
# '주거 형태'는 '월세', '주택 담보 대출 (거주 중)', '자가', '주택 담보 대출 (비거주 중)' 4개 범주 명목형 데이터 -> one hot 인코딩
# 주택 담보 대출 (비거주 중) 데이터 매우 불충분

# One-Hot 인코딩 수행
train_df = pd.get_dummies(train_df, columns=['주거 형태'])
test_df = pd.get_dummies(test_df, columns=['주거 형태'])

In [ ]:
# 피쳐 추가
features.extend(['주거 형태_월세', '주거 형태_자가', '주거 형태_주택 담보 대출 (거주 중)', '주거 형태_주택 담보 대출 (비거주 중)'])

In [45]:
# '현재 직장 근속 연수'는 '1년 미만', '1년', '2년', '3년', '4년', '5년', '6년', '7년', '8년', '9년', '10년 이상' 범주 순서형 데이터 -> 라벨 인코딩
# 10년 이상 데이터가 다수, 나머지 데이터 비율은 비슷
# 데이터 불균형

ordinal_mapping = {
    '1년 미만': 0,
    '1년': 1,
    '2년': 2,
    '3년': 3,
    '4년': 4,
    '5년': 5,
    '6년': 6,
    '7년': 7,
    '8년': 8,
    '9년': 9,
    '10년 이상': 10
}
train_df['근속 연수 라벨 인코딩'] = train_df['현재 직장 근속 연수'].map(ordinal_mapping)
test_df['근속 연수 라벨 인코딩'] = test_df['현재 직장 근속 연수'].map(ordinal_mapping)

In [46]:
# 피쳐 추가
features.append('근속 연수 라벨 인코딩')

In [48]:
# 대출 목적 부채통합의 비율 0.7294, 그 이외의 비율 0.2706
# 부채통합이면 1, 아니면 0 범주 명목형 데이터 커스텀 피쳐 추가

train_df['대출 목적_부채통합'] = train_df['대출 목적'].apply(lambda x: 1 if x == '부채통합' else 0)
test_df['대출 목적_부채통합'] = test_df['대출 목적'].apply(lambda x: 1 if x == '부채통합' else 0)

In [49]:
features.append('대출 목적_부채통합')

In [52]:
# 대출 상환 기간 데이터는 범주 명목형 데이터
# 단기 상환    0.6975, 장기 상환    0.3025
# '단기 상환'이면 0, '장기 상환'이면 1

train_df['대출 상환 범주'] = train_df['대출 상환 기간'].apply(lambda x: 0 if x == '단기 상환' else 1)
test_df['대출 상환 범주'] = test_df['대출 상환 기간'].apply(lambda x: 0 if x == '단기 상환' else 1)

In [53]:
features.append('대출 상환 범주')

In [54]:
# 나머지 수치형 데이터 비대칭적이다.
# 수치형 데이터 Transfomation -> Scaling 작업 필요
# Transform 방법, Scaling 방법 여러가지 경우의 수
# 베이스모델은 PowerTransformer yeo-johnson, MinMaxScaler

# 트랜스폼 적용 후 이상치 탐지
# 사용할 이상치: '최대 신용한도'
# '최대 신용한도_is_outlier' 생성 후 표시
# 원래 컬럼의 이상치를 중앙값으로 대체체
# 나머지 이상치들은 비율이 매우 적기 때문에 삭제해도 무방하다.
# 이상치 버릴거: 연간 소득, 개설된 신용계좌 수, 신용 거래 연수, 마지막 연체 이후 경과 개월 수, 현재 대출 잔액, 현재 미상환 신용액, 월 상환 부채액, 신용 점수
# 이후 MinMaxScaling

In [ ]:
# 데이터 yeo-johnson transform

columns = [
    '연간 소득', '개설된 신용계좌 수', '신용 거래 연수', '최대 신용한도',
    '마지막 연체 이후 경과 개월 수', '현재 대출 잔액', '현재 미상환 신용액',
    '월 상환 부채액', '신용 점수'
]

from sklearn.preprocessing import PowerTransformer
transfomer = PowerTransformer(method='yeo-johnson')

for i, column in enumerate(columns, 1):
    train_df[column] = transfomer.fit_transform(train_df[[column]])
    test_df[column] = transfomer.fit_transform(test_df[[column]])

In [ ]:
# train_df 이상치 제거 함수 정의
def remove_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

In [ ]:
# 최대 신용한도 이상치는 따로 처리
# 이상치 제거

outlier_columns = [
    '연간 소득', '개설된 신용계좌 수', '신용 거래 연수',
    '마지막 연체 이후 경과 개월 수', '현재 대출 잔액', '현재 미상환 신용액',
    '월 상환 부채액', '신용 점수'
]

for column in outlier_columns:
    train_df = remove_outliers(train_df, column)

In [61]:
# 사용할 이상치: '최대 신용한도'
# '최대 신용한도_is_outlier' 생성 후 표시
# 원래 컬럼의 이상치를 중앙값으로 대체

# 이상치 검출 및 표시 함수 정의 (train_df 기준으로)
def detect_outliers(df, column, lower_bound, upper_bound):
    df[column + '_is_outlier'] = df[column].apply(lambda x: 1 if x < lower_bound or x > upper_bound else 0)
    return df

# train_df에서 IQR 계산 및 이상치 기준 설정
Q1 = train_df['최대 신용한도'].quantile(0.25)
Q3 = train_df['최대 신용한도'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# 이상치 검출 및 표시
train_df = detect_outliers(train_df, '최대 신용한도', lower_bound, upper_bound)
test_df = detect_outliers(test_df, '최대 신용한도', lower_bound, upper_bound)

# 중앙값 계산 (train_df 기준으로)
median_value = train_df['최대 신용한도'].median()

# 이상치를 중앙값으로 대체
train_df['최대 신용한도'] = train_df.apply(lambda row: median_value if row['최대 신용한도_is_outlier'] == 1 else row['최대 신용한도'], axis=1)
test_df['최대 신용한도'] = test_df.apply(lambda row: median_value if row['최대 신용한도_is_outlier'] == 1 else row['최대 신용한도'], axis=1)

In [65]:
features.append('최대 신용한도_is_outlier')

In [67]:
# 스케일링
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

train_df[columns] = scaler.fit_transform(train_df[columns])
test_df[columns] = scaler.fit_transform(test_df[columns])

In [68]:
features.extend(columns)

In [72]:
features

['체납 세금 압류 여부',
 '신용 문제 범주',
 '개인 파산 범주',
 '주거 형태_월세',
 '주거 형태_자가',
 '주거 형태_주택 담보 대출 (거주 중)',
 '주거 형태_주택 담보 대출 (비거주 중)',
 '근속 연수 라벨 인코딩',
 '대출 목적_부채통합',
 '대출 상환 범주',
 '최대 신용한도_is_outlier',
 '연간 소득',
 '개설된 신용계좌 수',
 '신용 거래 연수',
 '최대 신용한도',
 '마지막 연체 이후 경과 개월 수',
 '현재 대출 잔액',
 '현재 미상환 신용액',
 '월 상환 부채액',
 '신용 점수']

In [73]:
# 베이스 모델 학습

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

X = train_df[features]
y = train_df[target]

# 데이터 분할 (학습 데이터와 테스트 데이터)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 모델 학습
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [74]:
# 예측 및 성능 평가
from sklearn.metrics import accuracy_score, classification_report

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy:.2f}')
print('Classification Report:')
print(report)

Accuracy: 0.71
Classification Report:
              precision    recall  f1-score   support

           0       0.72      0.90      0.80      1222
           1       0.65      0.35      0.45       654

    accuracy                           0.71      1876
   macro avg       0.68      0.62      0.63      1876
weighted avg       0.70      0.71      0.68      1876



In [75]:
y_submit = model.predict(test_df[features])
y_submit

array([0, 0, 0, ..., 0, 0, 0], shape=(2062,))

In [76]:
submit_df = pd.read_csv('./data/sample_submission.csv')

In [77]:
submit_df

,UID,채무 불이행 확률
0,TEST_0000,0
1,TEST_0001,0
2,TEST_0002,0
3,TEST_0003,0
4,TEST_0004,0
...,...,...
2057,TEST_2057,0
2058,TEST_2058,0
2059,TEST_2059,0
2060,TEST_2060,0


In [78]:
submit_df['채무 불이행 확률'] = y_submit

In [79]:
submit_df

,UID,채무 불이행 확률
0,TEST_0000,0
1,TEST_0001,0
2,TEST_0002,0
3,TEST_0003,0
4,TEST_0004,0
...,...,...
2057,TEST_2057,0
2058,TEST_2058,0
2059,TEST_2059,0
2060,TEST_2060,0


In [80]:
submit_df['채무 불이행 확률'].value_counts()

채무 불이행 확률
0    1961
1     101
Name: count, dtype: int64

In [82]:
submit_df.to_csv('./data/submission_02_19.csv', index=False)